In [1]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint

from sheet_id.models.FCN import FCN
from sheet_id.utils.loss_functions import softmax_sparse_crossentropy_ignoring_last_label
from sheet_id.utils.metrics import sparse_accuracy_ignoring_last_label
from sheet_id.utils.dataReader import seg_dataset_reader
from sheet_id.utils.dataPreprocessing import splitTrainValidation
from sheet_id.utils.dataGenerator import DataGenerator

Using TensorFlow backend.


In [2]:
model = FCN(input_shape=(500,500,1), n_classes=124)

In [3]:
loss_fn = softmax_sparse_crossentropy_ignoring_last_label
optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, 
                 epsilon=1e-8, decay=0.0, amsgrad=False)
metrics = [sparse_accuracy_ignoring_last_label]
model.compile(loss=loss_fn, optimizer=optimizer, metrics=metrics)

In [4]:
data_reader = seg_dataset_reader('../data/', max_pages=1, test_size=0,
                                 crop=True, crop_size=[500,500])

Initializing DeepScores Classification Batch Dataset Reader...
Splitting dataset, train: 1 images, test: 0 images
im working!7
Training set done
Test set done


In [5]:
callbacks = []

In [6]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=10, write_graph=True)
callbacks.append(tensorboard)

In [7]:
checkpoint = ModelCheckpoint(filepath='./checkpoint/checkpoint_weights.hdf5', 
                             save_weights_only=True, period=100) #.{epoch:d}
callbacks.append(checkpoint)

In [8]:
train_partition, validation_partition = splitTrainValidation('../data/', max_size=1, test_size=0)

In [9]:
params = {'dim': (500,500),
          'batch_size': 20,
          'n_classes': 124,
          'n_channels': 1,
          'shuffle': True,
          'crop': True,
          'crop_size': [500,500]}
training_generator = DataGenerator(train_partition, **params)
validation_generator = DataGenerator(train_partition, **params)

In [14]:
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True, workers=6)

Epoch 1/1
1/1 [==============================] - 58s 58s/step - loss: 18.1507 - sparse_accuracy_ignoring_last_label: 3.6000e-05 - val_loss: 9.3204 - val_sparse_accuracy_ignoring_last_label: 0.0094
